## 1. Importar Bibliotecas e Configuração

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from scipy.stats import spearmanr
from sklearn.metrics import (
    roc_auc_score, 
    average_precision_score,
    accuracy_score,
    precision_recall_fscore_support,
    matthews_corrcoef,
    confusion_matrix
)
import joblib
import torch
import warnings
warnings.filterwarnings('ignore')

# Configuração de visualização
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')
sns.set_context('notebook', font_scale=1.2)
%matplotlib inline

# Configuração de cores consistentes
MODEL_COLORS = {
    'MLP Centralizado': '#1f77b4',
    'MLP Federado': '#ff7f0e',
    'XGBoost Centralizado': '#2ca02c',
    'XGBoost Federado (Bagging)': '#d62728',
    'XGBoost Federado (Cyclic)': '#9467bd'
}

print("Bibliotecas importadas com sucesso!")

## 2. Definir Caminhos e Carregar Dados

In [ ]:
# Definir caminhos base
project_root = Path.cwd().parent
centralized_dir = project_root / 'centralized_training' / 'models'
flwr_mlp_dir = project_root / 'flwr-mlp' / 'models'
flwr_xgb_dir = project_root / 'flwr-xgboost' / 'models'
xai_dir = project_root / 'xAI'
datasets_dir = project_root / 'datasets'

# Verificar existência
print("Verificando estrutura de diretórios...")
for path in [centralized_dir, flwr_mlp_dir, flwr_xgb_dir, xai_dir, datasets_dir]:
    status = "OK" if path.exists() else "AUSENTE"
    print(f"  [{status}] {path.name}: {path}")

# Carregar índices de teste
test_indices_df = pd.read_csv(datasets_dir / 'test_indices.csv')
test_indices = test_indices_df['index'].values
print(f"\n{len(test_indices)} amostras de teste carregadas")
print(f"Colunas disponíveis: {list(test_indices_df.columns)}")

## 1.1. Curvas de Aprendizado (Convergência)

In [ ]:
# Carregar históricos de treinamento
print("Carregando históricos de treinamento...\n")

histories = {}

# MLP Centralizado
mlp_cent_path = centralized_dir / 'mlp' / 'training_history.csv'
if mlp_cent_path.exists():
    histories['MLP Centralizado'] = pd.read_csv(mlp_cent_path)
    print(f"[OK] MLP Centralizado: {len(histories['MLP Centralizado'])} épocas")
else:
    print(f"[AUSENTE] MLP Centralizado: NÃO ENCONTRADO")

# XGBoost Centralizado
xgb_cent_path = centralized_dir / 'xgboost' / 'training_history.csv'
if xgb_cent_path.exists():
    histories['XGBoost Centralizado'] = pd.read_csv(xgb_cent_path)
    print(f"[OK] XGBoost Centralizado: {len(histories['XGBoost Centralizado'])} iterações")
else:
    print(f"[AUSENTE] XGBoost Centralizado: NÃO ENCONTRADO")

# MLP Federado
mlp_fed_path = flwr_mlp_dir / 'training_history.csv'
if mlp_fed_path.exists():
    histories['MLP Federado'] = pd.read_csv(mlp_fed_path)
    print(f"[OK] MLP Federado: {len(histories['MLP Federado'])} rounds")
else:
    print(f"[AUSENTE] MLP Federado: NÃO ENCONTRADO")

print(f"\nTotal de modelos com histórico: {len(histories)}/5")

In [ ]:
# Plotar curvas de aprendizado
fig, axes = plt.subplots(1, 3, figsize=(20, 5))
fig.suptitle('Curvas de Aprendizado: Convergência dos Modelos', fontsize=16, fontweight='bold')

# MLP Centralizado
if 'MLP Centralizado' in histories:
    df = histories['MLP Centralizado']
    axes[0].plot(df['epoch'], df['val_aucpr'], 'o-', label='Val AUCPR', linewidth=2, markersize=4)
    axes[0].plot(df['epoch'], df['val_auc'], 's-', label='Val AUC', linewidth=2, markersize=4, alpha=0.7)
    axes[0].set_xlabel('Época', fontweight='bold')
    axes[0].set_ylabel('Métrica', fontweight='bold')
    axes[0].set_title('MLP Centralizado', fontweight='bold')
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)
    
    best_epoch = df['val_aucpr'].idxmax() + 1
    axes[0].axvline(x=best_epoch, color='red', linestyle='--', alpha=0.5, label=f'Best: {best_epoch}')

# XGBoost Centralizado
if 'XGBoost Centralizado' in histories:
    df = histories['XGBoost Centralizado']
    axes[1].plot(df['iteration'], df['val_aucpr'], 'o-', label='Val AUCPR', linewidth=2, markersize=4)
    axes[1].plot(df['iteration'], df['val_auc'], 's-', label='Val AUC', linewidth=2, markersize=4, alpha=0.7)
    axes[1].set_xlabel('Iteração (Árvore)', fontweight='bold')
    axes[1].set_ylabel('Métrica', fontweight='bold')
    axes[1].set_title('XGBoost Centralizado', fontweight='bold')
    axes[1].legend()
    axes[1].grid(True, alpha=0.3)
    
    best_iter = df['val_aucpr'].idxmax() + 1
    axes[1].axvline(x=best_iter, color='red', linestyle='--', alpha=0.5, label=f'Best: {best_iter}')

# MLP Federado
if 'MLP Federado' in histories:
    df = histories['MLP Federado']
    axes[2].plot(df['round'], df['val_aucpr'], 'o-', label='Val AUCPR', linewidth=2, markersize=4)
    axes[2].plot(df['round'], df['val_auc'], 's-', label='Val AUC', linewidth=2, markersize=4, alpha=0.7)
    axes[2].set_xlabel('Round', fontweight='bold')
    axes[2].set_ylabel('Métrica', fontweight='bold')
    axes[2].set_title('MLP Federado (FedAvg)', fontweight='bold')
    axes[2].legend()
    axes[2].grid(True, alpha=0.3)
    
    best_round = df['val_aucpr'].idxmax() + 1
    axes[2].axvline(x=best_round, color='red', linestyle='--', alpha=0.5, label=f'Best: Round {best_round}')

plt.tight_layout()
plt.savefig(project_root / 'notebooks' / 'fig1_curvas_aprendizado.png', dpi=300, bbox_inches='tight')
plt.show()

print("\nFigura 1 salva: fig1_curvas_aprendizado.png")

In [ ]:
# Análise quantitativa de convergência
print("="*80)
print("ANÁLISE DE CONVERGÊNCIA")
print("="*80)

for model_name, df in histories.items():
    print(f"\n{model_name}:")
    
    # Identificar coluna de iteração
    iter_col = 'epoch' if 'epoch' in df.columns else ('round' if 'round' in df.columns else 'iteration')
    
    # Melhor performance
    best_idx = df['val_aucpr'].idxmax()
    best_iter = df.loc[best_idx, iter_col]
    best_aucpr = df.loc[best_idx, 'val_aucpr']
    
    # Performance final
    final_aucpr = df['val_aucpr'].iloc[-1]
    
    # Degradação
    degradation = best_aucpr - final_aucpr
    degradation_pct = (degradation / best_aucpr) * 100
    
    # Rounds/épocas de overfitting
    overfit_iters = len(df) - (best_idx + 1)
    overfit_pct = (overfit_iters / len(df)) * 100
    
    print(f"  Melhor {iter_col}: {best_iter} (AUCPR: {best_aucpr:.6f})")
    print(f"  Final {iter_col}: {len(df)} (AUCPR: {final_aucpr:.6f})")
    print(f"  Degradação: {degradation:.6f} ({degradation_pct:.2f}%)")
    print(f"  Iterações pós-best: {overfit_iters}/{len(df)} ({overfit_pct:.1f}%)")
    
    if degradation > 0.01:
        print(f"  [ALERTA] Degradação significativa detectada")
    else:
        print(f"  [OK] Convergência estável")

## 1.2. Tabela de Métricas de Classificação

In [ ]:
print("Preparando análise de métricas de classificação...")
print("\n[INFO] Requer carregar modelos e fazer predições no conjunto de teste")
print("\nEsta célula será completada na próxima etapa com:")
print("  - Acurácia")
print("  - Precisão, Recall, F1-Score")
print("  - AUC-ROC")
print("  - AUC-PR (métrica principal)")
print("  - Matthews Correlation Coefficient (MCC)")

## 2.1. Comparação de Feature Importance (SHAP) + Correlação de Spearman

In [ ]:
# Carregar feature importance de todos os modelos
print("Carregando SHAP Feature Importance...\n")

shap_importance = {}

# Caminhos para cada modelo
shap_paths = {
    'MLP Centralizado': xai_dir / 'shap_results_centralized' / 'mlp' / 'feature_importance_all.csv',
    'MLP Federado': xai_dir / 'shap_results_federated' / 'mlp' / 'feature_importance_all.csv',
    'XGBoost Centralizado': xai_dir / 'shap_results_centralized' / 'xgboost' / 'feature_importance_all.csv',
    'XGBoost Federado (Bagging)': xai_dir / 'shap_results_federated' / 'xgboost' / 'bagging_strategy' / 'feature_importance_all.csv',
    'XGBoost Federado (Cyclic)': xai_dir / 'shap_results_federated' / 'xgboost' / 'cyclic_strategy' / 'feature_importance_all.csv',
}

for model_name, path in shap_paths.items():
    if path.exists():
        shap_importance[model_name] = pd.read_csv(path)
        print(f"[OK] {model_name}: {len(shap_importance[model_name])} features")
    else:
        print(f"[AUSENTE] {model_name}: NÃO ENCONTRADO")

print(f"\nTotal de modelos carregados: {len(shap_importance)}/5")

In [ ]:
# Plotar Feature Importance lado a lado
n_models = len(shap_importance)
fig, axes = plt.subplots(1, n_models, figsize=(6*n_models, 6))
if n_models == 1:
    axes = [axes]

fig.suptitle('Comparação de Feature Importance Global (SHAP)', fontsize=16, fontweight='bold')

for ax, (model_name, df) in zip(axes, shap_importance.items()):
    # Pegar top 15 features
    top_features = df.nlargest(15, 'Mean_Abs_SHAP')
    
    # Plotar barras horizontais
    ax.barh(range(len(top_features)), top_features['Mean_Abs_SHAP'], 
            color=MODEL_COLORS.get(model_name, 'gray'), alpha=0.8)
    ax.set_yticks(range(len(top_features)))
    ax.set_yticklabels(top_features['Feature'])
    ax.invert_yaxis()
    ax.set_xlabel('Mean |SHAP Value|', fontweight='bold')
    ax.set_title(model_name, fontweight='bold')
    ax.grid(axis='x', alpha=0.3)

plt.tight_layout()
plt.savefig(project_root / 'notebooks' / 'fig2_feature_importance_comparison.png', dpi=300, bbox_inches='tight')
plt.show()

print("\nFigura 2 salva: fig2_feature_importance_comparison.png")

In [ ]:
# Calcular Correlação de Spearman entre todos os pares de modelos
from itertools import combinations

print("="*80)
print("ANÁLISE DE CORRELAÇÃO DE SPEARMAN (ρ)")
print("="*80)
print("\nMedida de concordância entre rankings de feature importance\n")

# Preparar dados para correlação
model_names = list(shap_importance.keys())
spearman_results = []

for model1, model2 in combinations(model_names, 2):
    df1 = shap_importance[model1].set_index('Feature')['Mean_Abs_SHAP']
    df2 = shap_importance[model2].set_index('Feature')['Mean_Abs_SHAP']
    
    # Alinhar features (usar apenas features comuns)
    common_features = df1.index.intersection(df2.index)
    df1_aligned = df1[common_features]
    df2_aligned = df2[common_features]
    
    # Calcular Spearman
    rho, p_value = spearmanr(df1_aligned, df2_aligned)
    
    spearman_results.append({
        'Modelo 1': model1,
        'Modelo 2': model2,
        'Spearman ρ': rho,
        'p-value': p_value,
        'Features Comuns': len(common_features)
    })
    
    # Interpretar
    if rho >= 0.9:
        interpretation = "[EXCELENTE] Concordância MUITO ALTA"
    elif rho >= 0.7:
        interpretation = "[BOM] Concordância ALTA"
    elif rho >= 0.5:
        interpretation = "[MODERADO] Concordância MODERADA"
    else:
        interpretation = "[BAIXO] Concordância BAIXA"
    
    print(f"{model1} vs {model2}:")
    print(f"  ρ = {rho:.4f} (p < {p_value:.2e})")
    print(f"  {interpretation}\n")

# Criar DataFrame de resultados
spearman_df = pd.DataFrame(spearman_results)

# Salvar resultados
spearman_df.to_csv(project_root / 'notebooks' / 'spearman_correlation_results.csv', index=False)
print("\nResultados salvos: spearman_correlation_results.csv")

In [ ]:
# Visualizar matriz de correlação de Spearman
from scipy.cluster import hierarchy
from scipy.spatial.distance import squareform

# Criar matriz de correlação
n_models = len(model_names)
corr_matrix = np.eye(n_models)

for i, model1 in enumerate(model_names):
    for j, model2 in enumerate(model_names):
        if i < j:
            df1 = shap_importance[model1].set_index('Feature')['Mean_Abs_SHAP']
            df2 = shap_importance[model2].set_index('Feature')['Mean_Abs_SHAP']
            common_features = df1.index.intersection(df2.index)
            rho, _ = spearmanr(df1[common_features], df2[common_features])
            corr_matrix[i, j] = rho
            corr_matrix[j, i] = rho

# Plotar heatmap
fig, ax = plt.subplots(figsize=(10, 8))
sns.heatmap(corr_matrix, annot=True, fmt='.3f', cmap='RdYlGn', 
            xticklabels=[name.replace(' ', '\n') for name in model_names],
            yticklabels=model_names,
            vmin=0, vmax=1, center=0.7,
            cbar_kws={'label': 'Correlação de Spearman (ρ)'},
            ax=ax)
ax.set_title('Matriz de Correlação de Rankings (Feature Importance)', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.savefig(project_root / 'notebooks' / 'fig3_spearman_heatmap.png', dpi=300, bbox_inches='tight')
plt.show()

print("\nFigura 3 salva: fig3_spearman_heatmap.png")